In [1]:
import ee
import folium

from eegeolab.bmath import SAVI

In [2]:
ee.Initialize()

In [3]:
savi = SAVI(nir='B5', red='B4')
savi.__dict__

{'nir': 'B5', 'red': 'B4', 'L': 0.5, '_name': 'SAVI'}

In [4]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
lat, lon = 37.9018, -122.292
point = ee.Geometry.Point(lon, lat)

l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

image = ee.Image(
  l8.filterBounds(point)
    .filterDate('2015-01-01', '2015-12-31')
    .sort('CLOUD_COVER')
    .first()
)

In [6]:
# Calculate SAVI
compute = savi.compute(image=image)

In [7]:
my_map = folium.Map(location=[lat, lon], zoom_start=9)
vis = {'min': -1, 'max': 1, 'bands': ['SAVI'], 'palette': ['blue', 'white', 'green']}

# Add the land cover to the map object.
my_map.add_ee_layer(compute, vis, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)